# FIT5196 Task 3 in Assessment 1
#### Student Name: Syed Ali Alim Rizvi
#### Student ID: 28984773

Date: 08/04/2018

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* re: for regexes. 
* json: for file to be converted to json output file.

<font size=3, color="red"> Note: this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. It is important to make sure the logic of how you finish the assessment is clearly shown in this notebook! </font>

## 1.  Import libraries 

In [1]:
#lib for regular expression
# !pip install re
# !pip install json
import re
import json

## 2. Parse XML File

In this section, you can write your python scripts to parse the correspondiing file.
You should 
* write proper notes for all code block in this notebook using the Markdown cells
* provide proper comment in your scripts
* run all cells to make sure scripts are runable. If the scripts cannot be run by the assessors, they will not be assessed and zero mark will be given to the task.

In [2]:
# open the file
xml = open('australian-sport-thesaurus-student.xml', 'r', encoding='utf-8')

##### Regexes and a bit of tweaking

Since each element is within one line hence the text between the tags can be extracted using a regex that could be applied to that line. In this implementation we make use of the group feature in regexes to extract text from that specific group as the text for our variable. The regexes used as shown further below make use of the the amount of spaces before each tag aswell as the name of the tags. 

A seperate regex is created to extract text from each element i.e. a seperate regex for title, description, related terms etc. 

Once applied it can be seen that one of the elements of description is stored in seperate lines hence the regex created does not work as it also tries to match the end of the tag within the line. This issue is further explained below.

The description element that is spread across multiple lines is as follows.

```
<Description>Athlete with one of a wide range of visual impairment (Blindness).

Suggest: Use for [non preferred term] Blind Athlete</Description>
```

due to the fact that one line for <Description> doesnt end with the tag. hence we create two new regexes to store the description for the first error line and another regex to store the description for the second error line then concatinate the two results to store the corrected description in the final dictionary.
   
```python
desc = re.compile(r'^[ ]{4}<Description>(.[^>]*)</Description>$') #for normal desc
desc_error_start = re.compile(r'^[ ]{4}<Description>(.[^>]*)$') #for broken  desc start
desc_error_end = re.compile(r'^(.[^>]*)</Description>$') #for broken desc end
```

since in the regex `(.[^>]*)` is gready hence to avoid it from including `<Description>` in the text we add a `[^>]` check so that it does not include it. 

### Regexes

In [3]:
#regexes to use to extract
desc = re.compile(r'^[ ]{4}<Description>(.[^>]*)</Description>$') #for normal desc
desc_error_start = re.compile(r'^[ ]{4}<Description>(.[^>]*)$') #for broken  desc start
desc_error_end = re.compile(r'^(.[^>]*)</Description>$') #for broken desc end
title = re.compile(r'^[ ]{4}<Title>(.*)</Title>$') #for title
rel_title = re.compile(r'^[ ]{8}<Title>(.*)</Title>$') #for related title
rel_relationship = re.compile(r'^[ ]{8}<Relationship>(.*)</Relationship>$') #for related relationship

In [4]:
#open excel file to read using utf-8 encoding
xml = open('australian-sport-thesaurus-student.xml', 'r', encoding='utf-8')

dic = {'thesaurus' : []} #dictionary to hold final output
new_rec = {} #dictionary to store current record attributes
sorted_new_rec = {} #a dictionary to store sorted values of new_rec


sub_dic = False # a check to see whether the title attribute is for the subrecord or for the original

#loop over each line in exml
for line in xml:
    
    #check if a new data record starts; if yes put the rec attributes in final dictionary
    if line.startswith('  <Term>'):
        if new_rec: #if there has been a previous record put in the record after sorting the dictionary
            for key in sorted(new_rec):
                sorted_new_rec[key] = new_rec[key]
            dic['thesaurus'].append(sorted_new_rec)
            #empty both dictionaries for the next record attributes
            new_rec = {}
            sorted_new_rec = {}
        continue
    
    #if the line has the tag title check if the sub record tick is True or false. 
    elif '<Title>' in line:
        if sub_dic == False:
            rec_title = title.match(line)
            new_rec['Title'] = rec_title.group(1) 
        else:#if not subrecord add to rec dic as original title
            rec_rel_title = rel_title.match(line)
            #since related terms is a list and only has one element(dictionary) we use the index of 0 to get the dic and pass the key
            new_rec['RelatedTerms'][0]['Title'] = rec_rel_title.group(1) #if subrecord add to Related terms title in sub dic
    
    elif '<Description>' in line or '</Description>' in line: #check if description in line (some description elements dont include both tags)
        rec_desc = desc.match(line)
        try:
            new_rec['Description'] = rec_desc.group(1) #if tag is normal put in dictionary
        except AttributeError: #if error tag then
            if line.startswith('    <Description>'): #if starting of the error tag
                rec_desc = desc_error_start.match(line)
                desc_line = rec_desc.group(1)
                desc_line = desc_line.strip() #store the first part of the tag
            elif line.startswith('Suggest'): #if ending of the error tag
                rec_desc = desc_error_end.match(line)
                desc_line = desc_line + ' ' + rec_desc.group(1) #store the desc as the concatination of the two error ones
                new_rec['Description'] = desc_line #put in the tag desc in dictionary
    
    #when tag is of related terms then create a new sub dictionary by the key RelatedTerms to store its title and relationship
    elif '<RelatedTerms>' in line:
        new_rec['RelatedTerms'] = [{}]
        sub_dic = True
        continue
        
    #if related term tag ends check the sub dic as false
    elif '</RelatedTerms>' in line:
        sub_dic = False        
    
    elif '<Relationship>' in line:
        rec_rel_relationship = rel_relationship.match(line)
        new_rec['RelatedTerms'][0]['Relationship'] = rec_rel_relationship.group(1) #if tag is of relationship add it to related terms sub dic

#once extracted close the file handle
xml.close()

### Save as JSON

In [5]:
#store json data in file
file = json.dumps(dic)

#create a new file and write to it (over write if already exists)
fh = open("sport.dat","w")
fh.write(file)
fh.close()

In [6]:
#print(json.dumps(dic))

## 3. Summary
Give a short summary of your work done above, such as your findings.

Converting the xml to json was somewhat of an easy task. the challanging part was not the cleaning or the formating. Since both the file formats are similar. 

The challenging part was identifying how the data should finally be stored in json. Also to identify the pattern in order to make the regexes as well as the if statements to tell the code what to do in what scenario. 

One of the issues faced was that all of the elements were in seperate but enclosed in one line when the file was read in except for one specific description tag. For this a two new regexes had to be created to extract the text from multiple lines and concatinate them before finally storing them in the correct format for the json file. 